In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow_datasets as tfds
import os,cv2,keras
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tqdm import tqdm
import time
import random

from keras.layers import Dense
from keras import Model
from keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.applications.vgg16 import VGG16

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

from keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
#This function is used to get intersection over union between two bounding boxes. This is very crucial function in any rcnn object detection model.
def get_iou(bb1, bb2):
    assert bb1['x1'] < bb1['x2']  #These are just comparing coordinates of both the bounding boxes (bb1 & bb2)
    assert bb1['y1'] < bb1['y2']
    assert bb2['x1'] < bb2['x2']
    assert bb2['y1'] < bb2['y2']
    
    x_left = max(bb1['x1'], bb2['x1']  #This is used to decide the bounding box by getting the max of each coordinate, so that there won't be anything missing
    y_top = max(bb1['y1'], bb2['y1'])
    x_right = min(bb1['x2'], bb2['x2'])
    y_bottom = min(bb1['y2'], bb2['y2'])
    
    if x_right < x_left or y_bottom < y_top:
        return 0.0
    intersection_area = (x_right - x_left) * (y_bottom - y_top)
    bb1_area = (bb1['x2'] - bb1['x1']) * (bb1['y2'] - bb1['y1'])
    bb2_area = (bb2['x2'] - bb2['x1']) * (bb2['y2'] - bb2['y1'])
    iou = intersection_area / float(bb1_area + bb2_area - intersection_area)
    assert iou >= 0.0
    assert iou <= 1.0
    return iou

In [ ]:
#This essentially used to parse the annotation files and extract bounding box coordinates. The coordinates are then stored in boxes list
def get_rectangle(csv_annot):
    df = pd.read_csv(csv_annot)
    boxes=[]
    for row in df.iterrows():
        x1 = int(row[1][0].split(" ")[0])
        y1 = int(row[1][0].split(" ")[1])
        x2 = int(row[1][0].split(" ")[2])
        y2 = int(row[1][0].split(" ")[3])
        box={"x1":x1,"x2":x2,"y1":y1,"y2":y2}
        boxes.append(box)
    return boxes

In [ ]:
#This function essentially extracts the rectangular region from the original image based on the given coordinates 
#and resize the rectangular region to 224x224 pixels thereby maintaining uniformity
def rectangle_from_image(image,x,y,w,h):
    timage = image[y:y+h,x:x+w]
    resized = cv2.resize(timage,
                     (224,224),
                     interpolation = cv2.INTER_AREA)
    return resized

In [ ]:
max_proposal = 2000
max_samples = 30
IOU_treshold_plane = 0.7
IOU_treshold_background = 0.3

dataset_path = "/kaggle/input/airplane-rcnn/Images/Images"
annot="/kaggle/input/airplane-rcnn/Airplanes_Annotations/Airplanes_Annotations"

ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()

train_images=[]
train_labels=[]
images=os.listdir(dataset_path)

In [ ]:
#Preprocess the given dataset
img_name=random.choice(images)
img = cv2.imread(os.path.join(dataset_path,img_name))
plt.imshow(img)
csv_file = os.path.join(annot,str(img_name).split(".")[0]+".csv")
boxes=get_rectangle(csv_file)
for box in boxes:
    cv2.rectangle(img,(box['x1'],box['y1']),(box['x2'],box['y2']),(255,0,0), 2)
plt.figure()
plt.imshow(img)
  
cv2.setUseOptimized(True);
ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation() #Uses selective search algorithm provided by opencv lib
ss.setBaseImage(img)
ss.switchToSelectiveSearchFast() #Uses selective search fast mode
rects = ss.process()
imOut = img.copy()
for i, rect in (enumerate(rects)):
    x, y, w, h = rect
    cv2.rectangle(imOut, (x, y), (x+w, y+h), (0, 255, 0), 1, cv2.LINE_AA)
# plt.figure()
plt.imshow(imOut)

In [ ]:
#Preprocess the given dataset by using the above functions
for file in tqdm(images[:100], desc="creating dataset"): #tqdm is for progress bar
    try:
        box_values=[]
        image=cv2.imread(os.path.join(dataset_path,file))
        csv_path = os.path.join(annot,file.split(".")[0]+".csv")
        box_values.extend(get_rectangle(csv_path))
        ss.setBaseImage(image)
        ss.switchToSelectiveSearchFast()
        ssresults = ss.process()
        imout = image.copy()
        counter=flag=falsecounter=fflag=bflag = 0
        for e, result in  enumerate(ssresults):
            if e< max_proposal and flag==0:
                for a_box in box_values:
                    x,y,w,h = result
                    iou = get_iou(a_box,{"x1":x,"x2":x+w,"y1":y,"y2":y+h})
                    if counter < max_samples:
                        if iou > IOU_treshold_plane:
                            train_images.append(rectangle_from_image(image,x,y,w,h))
                            train_labels.append(1)
                            counter +=1
                    else:
                        fflag = 1
                    if falsecounter < max_samples:
                        if iou < IOU_treshold_background:
                            train_images.append(rectangle_from_image(image,x,y,w,h))
                            train_labels.append(0)
                            falsecounter += 1
                    else:
                        bflag = 1
                if fflag == 1 and bflag==1:
                    flag=1
    except Exception as e:
        print(e)
        continue

In [ ]:
vggmodel = VGG16(weights='imagenet', include_top=True) #Using vgg16 architecture

for layers in (vggmodel.layers)[:15]: #This loop iterates over the first 15 layers of the VGG16 model.
    print(layers)
    layers.trainable = False #freezing these layers, their weights won't be updated during training, and only the weights of the added layers will be trained.
    
X= vggmodel.layers[-2].output
predictions = Dense(1, activation="sigmoid")(X) #This line adds a Dense layer with one neuron and a sigmoid activation function
model_final = Model(vggmodel.input,predictions)
opt = Adam()
model_final.compile(loss = keras.losses.binary_crossentropy, optimizer = opt, metrics=["accuracy"]) #Uses Adam optimizer and accuracy as it's metric for evaluating the performance
model_final.summary()

In [ ]:
X_new = np.array(train_images)
y_new = np.array(train_labels)

X_train, X_test , y_train, y_test = train_test_split(X_new,y_new,test_size=0.10)

trdata    = ImageDataGenerator(horizontal_flip=True, vertical_flip=True, rotation_range=0)
traindata = trdata.flow(x=X_train, y=y_train)
tsdata    = ImageDataGenerator(horizontal_flip=True, vertical_flip=True, rotation_range=0)
testdata  = tsdata.flow(x=X_test, y=y_test)

In [ ]:
#This code snippet sets up checkpoints and early stopping for the training process.
checkpoint = ModelCheckpoint("ieeercnn_vgg16_1.keras",
                             monitor='val_loss',
                             verbose=1,
                             save_best_only=True,
                             save_weights_only=False,
                             mode='auto')

early = EarlyStopping(monitor='val_loss', 
                      min_delta=0,
                      patience=100,
                      verbose=1,
                      mode='auto')

hist = model_final.fit(x= traindata,
                                 steps_per_epoch= 10,
                                 epochs= 4, #Ideally it shoulld be 100
                                 validation_data= testdata,
                                 validation_steps=2,
                                 callbacks=[checkpoint,early])

In [ ]:
img=random.choice(images)
img = cv2.imread(os.path.join(dataset_path,img))
ss.setBaseImage(img)
ss.switchToSelectiveSearchFast()
ssresults = ss.process()
imout = img.copy()
for e,result in enumerate(ssresults):
    if e < 2000:
        x,y,w,h = result
        timage = imout[y:y+h,x:x+w]
        resized = cv2.resize(timage, (224,224), interpolation = cv2.INTER_AREA)
        img = np.expand_dims(resized, axis=0)
        out= model_final.predict(img)
        if out[0][0] > 0.43: #Ideally threshold should be 0.7
            cv2.rectangle(imout, (x, y), (x+w, y+h), (0, 255, 0), 1, cv2.LINE_AA)
plt.figure()
plt.imshow(imout)